In [1]:
!pip install -q gradio  elevenlabs twilio requests
!pip install fal-client

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 26.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 92.1 MB/s eta 0:00:00


In [ ]:
import os
from openai import OpenAI

os.environ["ELEVENLABS_API_KEY"] = "API"
os.environ["TWILIO_SID"] = "API"
os.environ["TWILIO_TOKEN"] = "API"


In [ ]:
import uuid
import os
import requests
from pathlib import Path
import gradio as gr
from elevenlabs.client import ElevenLabs
from twilio.rest import Client as TwilioClient
from diffusers import StableDiffusionPipeline
import torch
import time

GROQ_API_KEY = "API"
GROQ_URL = "URL"
MODEL_ID = "ID"

history, favorites = [], []
out_dir = Path("excuse_outputs"); out_dir.mkdir(exist_ok=True)

device = "cuda" if torch.cuda.is_available() else "cpu"
sd_pipe = StableDiffusionPipeline.from_pretrained("runwayml/stable-diffusion-v1-5").to(device)

def call_groq(prompt):
    try:
        headers = {"Authorization": f"Bearer {GROQ_API_KEY}"}
        data = {
            "model": MODEL_ID,
            "messages": [
                {"role": "system", "content": "You are a polite assistant that writes excuses and apologies."},
                {"role": "user", "content": prompt}
            ],
            "max_tokens": 120,
            "temperature": 0.7
        }
        res = requests.post(GROQ_URL, headers=headers, json=data).json()
        if "choices" in res:
            return res["choices"][0]["message"]["content"].strip()
        elif "error" in res:
            return f"Error: {res['error']['message']}"
        else:
            return str(res)
    except Exception as e:
        return str(e)

def gen_excuse(scenario, context, urgency):
    prompt = f"Write a short, polite excuse for {scenario}. Urgency: {urgency}. Context: {context}."
    text = call_groq(prompt)
    history.append(text)
    return text

def gen_apology(tone, person, context):
    prompt = f"Write a {tone} apology to {person}. Reason: {context}."
    text = call_groq(prompt)
    history.append(text)
    return text

def to_audio(text, voice):
    text = text.strip() if text else ""
    if not text:
        return None, "Please generate an excuse first", None
    try:
        eleven_client = ElevenLabs(api_key=os.environ["ELEVENLABS_API_KEY"])
        audio_bytes = eleven_client.text_to_speech.convert(
            text=text,
            voice_id="21m00Tcm4TlvDq8ikWAM",
            model_id="eleven_multilingual_v2",
            output_format="mp3_44100_128"
        )
        audio_bytes = b"".join(audio_bytes)
        path = out_dir / f"excuse_{uuid.uuid4().hex[:6]}.mp3"
        with open(path, "wb") as f:
            f.write(audio_bytes)
        return str(path), "Audio ready", str(path)
    except Exception as e:
        return None, str(e), None

def send_sms(from_no, to_no, msg):
    msg = msg.strip() if msg else ""
    if not msg:
        return "No message to send"
    try:
        twilio_client = TwilioClient(os.environ["TWILIO_SID"], os.environ["TWILIO_TOKEN"])
        message = twilio_client.messages.create(body=msg, from_=from_no, to=to_no)
        return f"Sent: {message.sid}"
    except Exception as e:
        return str(e)


def gen_image(img_prompt, excuse_text):
    img_prompt = img_prompt.strip() if img_prompt else ""
    excuse_text = excuse_text.strip() if excuse_text else ""

    if img_prompt:
        prompt = img_prompt
    elif excuse_text:
        prompt = f"An expressive illustration of this excuse: {excuse_text}"
    else:
        return None

    try:
        image = sd_pipe(prompt, num_inference_steps=30).images[0]
        path = out_dir / f"excuse_{uuid.uuid4().hex[:6]}.png"
        image.save(path)
        return str(path)
    except Exception as e:
        return None


def save_fav(text):
    if text: favorites.append(text); return "Saved"
    return "No text"

def show_hist(): return "\n\n".join(history[-10:]) if history else "Empty"
def show_favs(): return "\n\n".join(favorites) if favorites else "Empty"


with gr.Blocks() as demo:
    gr.Markdown("## Intelligent Excuse and Apology Generator")

    with gr.Tab("Generate Excuse"):
        gr.Markdown("### Generate Excuse")
        scenario = gr.Dropdown(["Work","School","Social","Family","Other"], value="Work", label="Scenario")
        urgency = gr.Dropdown(["Low","Medium","High"], value="Medium", label="Urgency")
        context = gr.Textbox(label="Context")
        btn_exc = gr.Button("Generate Excuse")
        excuse_out = gr.Textbox(label="Generated Excuse", lines=3)
        btn_exc.click(fn=gen_excuse, inputs=[scenario, context, urgency], outputs=[excuse_out])

        gr.Markdown("### Excuse Actions")
        voice = gr.Dropdown(["default"], value="default", label="Voice")
        btn_audio = gr.Button("Convert to Audio")
        audio_p = gr.Audio(label="Audio Output")
        audio_status = gr.Textbox(label="Audio Status")
        audio_file = gr.File(label="Audio File")
        btn_audio.click(fn=to_audio, inputs=[excuse_out, voice], outputs=[audio_p, audio_status, audio_file])

        from_no = gr.Textbox(label="From Phone Number")
        to_no = gr.Textbox(label="To Phone Number")
        btn_sms = gr.Button("Send SMS")
        sms_status = gr.Textbox(label="SMS Status")
        btn_sms.click(fn=send_sms, inputs=[from_no, to_no, excuse_out], outputs=[sms_status])

        btn_fav = gr.Button("Save Excuse to Favorites")
        fav_status = gr.Textbox(label="Save Status")
        btn_fav.click(fn=save_fav, inputs=[excuse_out], outputs=[fav_status])

    with gr.Tab("Generate Apology"):
        gr.Markdown("### Generate Apology")
        tone = gr.Dropdown(["Sincere","Formal","Casual"], value="Sincere", label="Tone")
        person = gr.Textbox(label="To")
        ap_ctx = gr.Textbox(label="Context")
        btn_ap = gr.Button("Generate Apology")
        ap_out = gr.Textbox(label="Generated Apology", lines=3)
        btn_ap.click(fn=gen_apology, inputs=[tone, person, ap_ctx], outputs=[ap_out])

    with gr.Tab("History and Favorites"):
        gr.Markdown("### History and Favorites")
        btn_hist = gr.Button("Show History")
        hist_txt = gr.Textbox(lines=6, label="History")
        btn_hist.click(fn=show_hist, outputs=[hist_txt])

        btn_favs = gr.Button("Show Favorites")
        fav_txt = gr.Textbox(lines=6, label="Favorites")
        btn_favs.click(fn=show_favs, outputs=[fav_txt])

    with gr.Tab("Generate Image from Excuse"):
        gr.Markdown("### Generate Image from Excuse")
        img_prompt = gr.Textbox(label="Image Prompt (Uses Generated Excuse if Empty)")
        btn_img = gr.Button("Generate Image")
        img_out = gr.Image(type="filepath", label="Generated Image")


        btn_img.click(fn=gen_image, inputs=[img_prompt, excuse_out], outputs=[img_out])

demo.launch()

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model_index.json:   0%|          | 0.00/541 [00:00<?, ?B/s]

Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

preprocessor_config.json:   0%|          | 0.00/342 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

...

config.json:   0%|          | 0.00/547 [00:00<?, ?B/s]

vae/diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

unet/diffusion_pytorch_model.safetensors:   0%|          | 0.00/3.44G [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://79b24626faf89d7cc0.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
